##### Imports

In [1]:
%matplotlib inline
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import json
from binance import AsyncClient, DepthCacheManager, BinanceSocketManager
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import time

##### Atenção
 - Altere o arquivo API_CREDENTIALS template.json para API_CREDENTIALS.json
 - Insira suas credenciais geradas na Binance

In [2]:
file = open('API_CREDENTIALS.json',)
API_CREDENTIALS = json.load(file)

api_key = API_CREDENTIALS['api_key']
api_secret = API_CREDENTIALS['api_secret']

client = Client(api_key, api_secret)

###### Defining Functions

In [3]:
def four_days_graph(ativo):

    klines = client.get_historical_klines(ativo, AsyncClient.KLINE_INTERVAL_1MINUTE, "4 days ago UTC")
    klines_close = [[kline[0],kline[4]] for kline in klines]
    first = klines_close[0][0]
    for item in klines_close:
        item[0] = item[0] - first
        item[0] = item[0] / 60000

    np_klines_close = np.array(klines_close)
    df_pandas = pd.DataFrame(np_klines_close, columns=['minute', 'price'])

    df_pandas['minute'] = df_pandas['minute'].astype(float)
    df_pandas['price'] = df_pandas['price'].astype(float)

    X = np.array(df_pandas['minute'])
    X = X[:, np.newaxis]
    y = np.array(df_pandas['price'])
    y = y[:, np.newaxis]

    model = LinearRegression().fit(X, y)

    df_pandas['y_'] = model.intercept_ + model.coef_ * X
    df_pandas['error'] = (df_pandas['price'] - df_pandas['y_'])

    mean = df_pandas['error'].mean()
    std = df_pandas['error'].std()
    df_pandas['error_normalized'] = (df_pandas['error'] - mean)/std

    a = np.array(df_pandas[((df_pandas['error_normalized'] > -3) & (df_pandas['error_normalized'] < 3)) == False]['minute'])
    b = np.array(df_pandas[((df_pandas['error_normalized'] > -3) & (df_pandas['error_normalized'] < 3)) == False]['price'])

    df_pandas['intercept_'] =  df_pandas['price']-(df_pandas['minute']*model.coef_[0][0])

    max = df_pandas[df_pandas['intercept_'] == df_pandas['intercept_'].max()][['minute','price']]
    max = [max.iloc[0]['minute'], max.iloc[0]['price']]
    top_x = np.array(df_pandas['minute'])
    intercept = max[1]-(max[0]*model.coef_)
    top_y = model.coef_*top_x + intercept
    top_y = top_y[0][:, np.newaxis]

    min = df_pandas[df_pandas['intercept_'] == df_pandas['intercept_'].min()][['minute','price']]
    min = [min.iloc[0]['minute'], min.iloc[0]['price']]
    bottom_x = np.array(df_pandas['minute'])
    intercept = min[1]-(min[0]*model.coef_)
    bottom_y = model.coef_*bottom_x + intercept
    bottom_y = bottom_y[0][:, np.newaxis]

    #plt.figure(figsize=[16,9])
    #plt.plot(df_pandas['minute'], df_pandas['price'], label=ativo)
    #plt.plot(X, df_pandas['y_'], label="lin reg")
    #plt.plot(top_x, top_y)
    #plt.plot(bottom_x, bottom_y)
    #plt.scatter(a, b, color='red')

    #plt.title(ativo+" Price  [ÚLTIMAS 24H]", fontsize=18)
    #plt.ylabel("price", fontsize=12)
    #plt.xlabel("minutes", fontsize=12)
    #plt.show()

    deltaNoPeriodo = model.coef_ *(df_pandas.iloc[-1]['minute'])
    deltaNoPeriodo = deltaNoPeriodo[0][0]

    intercept = model.intercept_

    maximo =  df_pandas[df_pandas['price'] == df_pandas['price'].max()][['price']]
    maximo =  maximo.iloc[0]['price']

    minimo =  df_pandas[df_pandas['price'] == df_pandas['price'].min()][['price']]
    minimo =  minimo.iloc[0]['price']

    amplitude = maximo - minimo

    return_dict = {
      "deltaNoPeriodo": deltaNoPeriodo,
      "coef_": model.coef_,
      "intercept": intercept,
      "mean_error": mean,
      "std_error": std,
      "maximo": max[1],
      "amplitude": amplitude,
      "movimentacao": deltaNoPeriodo/maximo,
      "Z_score_error": 3,
      "first_minute": first,
      "last_minute": df_pandas.iloc[-1]["minute"]
    }

    return return_dict

In [5]:
def tendencias(ativo):
    four_days_dict = four_days_graph(ativo)

    tendencia_four_days = "A DEFINIR"
    if four_days_dict['movimentacao'] > 0.01:
        tendencia_four_days = "ALTA"
    elif four_days_dict['movimentacao'] < -0.01:
        tendencia_four_days = "BAIXA"
    else:
        tendencia_four_days = "LATERAL"

    print("TENDÊNCIA DE 4 DIAS É "+tendencia_four_days)


    return tendencia_four_days

###### Trading Thread

In [7]:
# SETS
op_list = []
sellSignal = 0
buySignal = 0
comprado = False
vendido = False

ultimoCandle = []


# START
# Calcular regressões 2 meses e um dia
tendencia_four_days = tendencias("ADAUSDT")
four_days_dict = four_days_graph("ADAUSDT")

noError = True

while(noError):
  # A CADA MINUTO:
  # Olhar o preço do Ativo
    penultimoCandle = ultimoCandle = []
    while(penultimoCandle == ultimoCandle):
        ultimoCandle = client.get_historical_klines("ADAUSDT", AsyncClient.KLINE_INTERVAL_1MINUTE, "1 minute ago UTC")
    ultimoCandle = [ultimoCandle[0][0], float(ultimoCandle[0][4])]
    currentMinute = ultimoCandle[0] / 60000
    ultimoCandle[0] = ultimoCandle[0] - four_days_dict["first_minute"]
    ultimoCandle[0] = ultimoCandle[0] / 60000
    i = ultimoCandle[0]

    yLinReg_lastCandle = four_days_dict["coef_"] * ultimoCandle[0] + four_days_dict["intercept"]
    error_lastCandle = (ultimoCandle[1] - yLinReg_lastCandle)
    zScore_lastCandle = (error_lastCandle - four_days_dict["mean_error"])/four_days_dict["std_error"]

    if(not comprado and not vendido):
        if((zScore_lastCandle > four_days_dict["Z_score_error"]) and (tendencia_four_days == "BAIXA")): # and (tendencia_two_months != "ALTA")):
            print("SINAL DE VENDA")
            sellSignal = 1

    if((sellSignal == 1)):
        # BUYY
        try:
            buy_order_market = client.create_order(symbol='ADAUSDT',
                                                   side='BUY',
                                                   type='MARKET',
                                                   quantity=30)
        except:
            noError = False
            
        f = open("OpAberta.txt", "w")
        f.write(str(buy_order_market))
        f.close()
        buy_price = ultimoCandle[1]
        comprado = True
        diff1 = ultimoCandle[1] - yLinReg_lastCandle 
        TP = yLinReg_lastCandle - 0.4*diff1 #- 0.4*diff1  #0.5 * four_days_dict["std_error"]
        diff = (ultimoCandle[1] - TP)
        STOP = ultimoCandle[1] + diff/2
        sellSignal = 0
        print("COMPRANDO")
        TP = TP[0][0]
        STOP = STOP[0][0]
        
        aux = TP
        TP = STOP
        STOP = aux
        
        f = open("OpAberta.txt", "a")
        f.write("TP:",TP,"STOP:",STOP)
        f.close()
        
        op_time = currentMinute

    if(comprado and ((ultimoCandle[1] >= TP) or (ultimoCandle[1] <= STOP))):
        # Sai da Operacao
        try:
            sell_order_market = client.create_order(symbol='ADAUSDT',
                                           side='BUY',
                                           type='MARKET',
                                           quantity=29.97)
        except:
            noError = False
            
        saida_price = ultimoCandle[1]
        if(saida_price >= buy_price):
                situacao = "GAIN"
                lucro = saida_price - buy_price
        if(saida_price < buy_price):
                situacao = "LOSS"
                lucro = saida_price - buy_price
        op_list.append([situacao,currentMinute - op_time, lucro, "BUY", buy_price, saida_price, TP, STOP, 30])
        f = open("listaOps.txt", "a")
        f.write(str([situacao,currentMinute - op_time, lucro, "BUY", buy_price, saida_price, TP, STOP, 30]))
        f.close()
        f = open("OpAberta.txt", "w")
        f.write("")
        f.close()

        comprado = False
        print("SAINDO DA OPERACAO DE COMPRA")

    if(i % 210 == 0):
        tendencia_four_days = tendencias("ADAUSDT")
        four_days_dict = four_days_graph("ADAUSDT")
        sellSignal = 0
        
    time.sleep(60)
    print("Proximo minuto")
        
    #if(i % 1440 == 0):
    #    tendencia_two_months = tendencias_2m_bt("ADAUSDT", lastPeriodInMins[i - 86400:i])
    #    sellSignal = 0
    #    buySignal = 0
#SLEEP 1 MIN


# se for uma anomalia e a operação estiver alinhada com a tendência, entrar em operação
# caso não, esperar o próximo minuto

# Caso tenha entrado na operação
# Definir o intervalo de preço p/ permanecer na operação
# Se sair do intervalo, realizar operação inversa

# A CADA HORA:
# Refazer a Regressão


TENDÊNCIA DE 4 DIAS É ALTA
Proximo minuto
Proximo minuto
Proximo minuto


KeyboardInterrupt: 